# 背水一战：全量数据QQA & Transformer_big

---

In [1]:
import os
import re
from tqdm import tqdm
import sys
import random
import pprint

sys.path.insert(0, "/home/team55/notespace/zengbin")

from jddc.config import Seq2SeqConfig
import jddc.utils as u
import jddc.datasets as d

base_conf = Seq2SeqConfig()
logger = u.create_logger(name='seq2seq', log_file=base_conf.log_file, cmd=base_conf.cmd_log)

In [2]:
conf = u.AttrDict()
conf['t2t_big_path'] = os.path.join(base_conf.base_path, "t2t_big")
conf['file_train'] = os.path.join(conf.t2t_big_path, "qa_pairs.tsv")
conf['file_train_rq'] = os.path.join(conf.t2t_big_path, "qa_pairs_rq.tsv")

In [3]:
import transformer_big.main as t
# import transformer_big.predict as p

temp file will saved in /home/team55/notespace/zengbin/transformer_big


## 构造训练集
---

In [ ]:
all_sessions = u.read_from_pkl(base_conf.pkl_sessions)

In [ ]:
right = all_sessions[:850000]
left = all_sessions[850000:]

In [ ]:
questions1, answers1 = d.create_dataset05(right, sample_n=800000, random_sample=False)
questions2, answers2 = d.create_dataset06(left, sample_n=100000)
questions = questions1 + questions2
answers = answers1 + answers2

In [ ]:
# 分词，保存成tsv
train_data = []
train_data_reverse_q = []
for i in tqdm(range(len(questions))): 
    q_tokens = u.jieba_tokenize(questions[i].replace("\n", ""))
    a_tokens = u.jieba_tokenize(answers[i].replace("\n", ""))
    row = " ".join(q_tokens) + "\t" + " ".join(a_tokens) 
    train_data.append(row)
    # 翻转src
    row_reverse_q = " ".join(q_tokens[::-1]) + "\t" + " ".join(a_tokens) 
    train_data_reverse_q.append(row_reverse_q)

u.write_file(conf.file_train, content=train_data, mode='w', encoding="utf-8")
u.write_file(conf.file_train_rq, content=train_data_reverse_q, mode='w', encoding="utf-8")

## 预测结果

---

In [ ]:
# 调用命令行预测
# t.run_prediction(base_conf.file_test_q, "/home/team55/notespace/zengbin/answers/transformer_big_answers028.txt", clear=True)

In [8]:
test_q = t.read_questions(base_conf.file_test_q)

In [4]:
from tensor2tensor import problems
import tensorflow as tf
import numpy as np
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
# 导入problem定义
from transformer_big import jddc_big

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys


In [5]:
# Setup some directories
data_dir = '/home/team55/notespace/data/t2t_big/data'
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_big"
checkpoint_dir = '/home/team55/notespace/data/t2t_big/model'
ckpt_path = tf.train.latest_checkpoint(checkpoint_dir)
print("ckpt:", ckpt_path)

problem_name = 'jddc_big'
jddc_problem = problems.problem(problem_name)
# Get the encoders from the problem
encoders = jddc_problem.feature_encoders(data_dir)


hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir,
                                     problem_name=problem_name)
jddc_model = registry.model(model_name)(hparams, Modes.EVAL)


ckpt: /home/team55/notespace/data/t2t_big/model/model.ckpt-124000
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0


In [15]:
# Setup helper functions for encoding and decoding
def encode(input_str):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}


def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))


def predict_one(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = jddc_model.infer(encoded_inputs)["outputs"]
    return decode(model_output)



In [36]:
answers = []
for i, q in enumerate(test_q):
    print(i, "=" * 88)
    q_pred = " ".join(t.jieba_tokenize(q))
    print("question:", q_pred)
    answer = predict_one(q_pred)
    answers.append(answer+"\n")
    print("answer:", answer)

0 ========================================================================================
question: 其他 问题 < q > 我 直接 解绑 了 实名 认证 的 那个 账号 ， 需要 把 手机号 也 解绑 了   是 的   金融 客服 说 这个 要 找 你们 解决   他们 不管
answer: 您 是 要 解绑 吗
1 ========================================================================================
question: ok < q > 新 电话号码 + [ 电话 x ]
answer: 好 的   请问 还有 其他 还 可以 帮到 您 的 吗 ?
2 ========================================================================================
question: [ 数字 x ]   这个 订单 的 发票 什么 时候 开 呢 < q > [ 数字 x ]   这 两个 订单号
answer: 发票 类型 ， 增值税 发票   发票 状态 ， 已   开
3 ========================================================================================
question: 我们 家 下午 六点 取件 ， 师傅 说 [ 数字 x ] 点 他们 取 不了 < q > 我 想 把 这个 退 了 ， 重新 买个
answer: 好 的   这边 帮 您 联系 站点
4 ========================================================================================
question: 你好 ， 这个 服务 单有 什么 问题 吗 ?   售后服务 单 号 : [ 数字 x ] < q > 我 电话 没响 过   麻烦 通知 客服 再 打 一遍
answer: 好 的   这边 帮 您 备注
5 ======================

answer: [ ORDERID _ 10002085 ]   这个 订单 吗
38 ========================================================================================
question: 是 的 < q > 有 电话 联系   但是 不是 物流 信息 上面 那个 电话   打过 我 一个 电话   然后 说 已经 安排 人 送货 了   但 没人 联系 我 收货
answer: [ ORDERID _ 10002085 ]   这个 订单 吗
39 ========================================================================================
question: 你好   我 这个 单 的 发票 之前 说 开过 了 但是 我 这边 收货 没 看到   上次 说 给 我 重新 发 一张 ， 现在 什么 结果 了 ? < q > 你好   这个 单据 寄往 哪里   你 帮 我 看戏   下
answer: [ ORDERID _ 10002085 ]   这个 订单 吗
40 ========================================================================================
question: 我 这 是 第一次 在 京东 退货 < q > 哦 。 好 的
answer: 请问 还有 其他 还 可以 帮到 您 的 吗 ?
41 ========================================================================================
question: 合并 开发票 ， 只能 按 订单 一页 一页 地点 吗 ? < q > 对 的   我 是 要 开 专票 的 ， 当时 下单 时 已经 填写 专票 信息
answer: 是 的 呢   您 的 订单 是 货到付款 现金 / 刷卡 支付 的 订单 ， 财务 需要 先 对账 的 ， 通常 需要 [ 数字 x ] - [ 数字 x ] 个 工作日 ， 对账 完成 后 ， 订单 会 变成 完成 状态 ， 

answer: 亲爱 的 ， 您 是 需要 修改 什么 呢
77 ========================================================================================
question: 之前 申请 入驻 的 店铺 类型 为 专营店     后来 因为 种种原因 没 通过 ， 现在 改 申请 旗舰店   品牌 也 换 了   系统 一直 提示 不能 更换 主营 类目 < q > 等于 这个 京东 账号 申请 过 店铺 就算 没 成功     也 是 没有 机会 申请 其他 品牌 的 店铺 了
answer: 您 是 要 修改 什么 呢
78 ========================================================================================
question: 等于 这个 京东 账号 申请 过 店铺 就算 没 成功     也 是 没有 机会 申请 其他 品牌 的 店铺 了 < q > [ 组织 机构 x ]
answer: 好 的   请问 还有 什么 可以 帮 您 呢 ? # E - s [ 数字 x ] 怠慢 之处 请 您 谅解 呢 # E - s [ 数字 x ]
79 ========================================================================================
question: [ 组织 机构 x ] < q > 没有 在 咱们 平台 开 过 店铺   之前 后台 申请 的 记录 确 删除 不了
answer: 亲爱 的 ， 您 是 什么 时候 申请 的 呢
80 ========================================================================================
question: 没有 在 咱们 平台 开 过 店铺   之前 后台 申请 的 记录 确 删除 不了 < q > 现在 要 申请 其他 品牌 的 店铺   现在 要 放弃     申请 其他 品牌 店铺
answer: 亲爱 的 ， 您 是 什么 时候 申请 的 呢
81 ======

In [41]:
answers_ = [x.replace(" ", "") for x in answers]
answers_

['您是要解绑吗\n',
 '好的请问还有其他还可以帮到您的吗?\n',
 '发票类型，增值税发票发票状态，已开\n',
 '好的这边帮您联系站点\n',
 '好的这边帮您备注\n',
 '[ORDERID_10002085]这个订单吗\n',
 '不会的呢\n',
 '您是说上门取件吗\n',
 '是的呢您的资质通过了吗\n',
 '亲爱哒方便提供一下订单号码吗小妹好去帮您查询\n',
 '[ORDERID_10002085]这个订单吗\n',
 '好的请问还有其他还可以帮到您的吗?\n',
 '您的订单预计[数字x]月[数字x]日送达您手中\n',
 '您的订单是货到付款现金/刷卡支付的订单，财务需要先对账的，通常需要[数字x]-[数字x]个工作日，对账完成后，订单会变成完成状态，[数字x]4H后系统自动生成电子发票的~~\n',
 '您的订单拦截成功，财务正在进行退款审核，请耐心等待。，\n',
 '#E-s[数字x]#E-s[数字x]请问还有什么可以帮助您的吗?\n',
 '可以的呢\n',
 '不客气的哦，分内之事哦，为您服务是我的荣幸哈#E-s[数字x]亲爱哒，请问还有什么可以帮您?#E-s[数字x]怠慢之处请您谅解呢#E-s[数字x]\n',
 '您是要开通京东物流吗\n',
 '您的订单预计[数字x]月[数字x]日送达您手中\n',
 '您的订单已经开始配送，系统正在尝试拦截，请耐心等待。，\n',
 '亲亲，您可以选择安装服务的\n',
 '[ORDERID_10002084]这个订单吗\n',
 '请问还有其他还可以帮到您的吗?\n',
 '您的订单预计[数字x]月[数字x]日送达您手中\n',
 '好的这边帮您联系站点核实您看可以吗\n',
 '[数字x]这个是站点的电话哦\n',
 '正在为您查询，还请您稍等两到三分钟的呢~#E-s[数字x]亲爱的，可以的呢\n',
 '#E-s[数字x]#E-s[数字x]请问还有其他可以帮到您的吗?\n',
 '您客气了，都是我们应该的#E-s[数字x]请问还有其他还可以帮到您的吗?\n',
 '提供绑定的银行卡卡号，后[数字x]位#E-s[数字x]\n',
 '是的呢亲亲#E-s[数字x]\n',
 '是的呢您下单的时候选择发票类型就可以了呢\n',
 '好的稍等您的订单拦截成功，财务正在进行退款审核，请

In [42]:
file_res = os.path.join(base_conf.base_path, "answers_temp_0918.txt")
with open(file_res, 'w', encoding='utf-8') as f:
    f.writelines(answers_)